
#Over here we require only one csv file 04_c 

CSV files 05_c, 06_c, 07_c are created using faker

In [ ]:
from datetime import datetime
import uuid
import random
import pandas as pd

In [ ]:
!pip show faker

In [ ]:
!pip install faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.2 MB/s 


In [ ]:
from faker import Faker
faker = Faker()

Making the User infos in the form of Functions so as to call these functions whenever required

In [ ]:
def get_age_years_month(dob):
  age_days = (datetime.now().date() - dob)
  age_days = age_days.days
  age_years = int(age_days/365)
  age_months = int((age_days%365)/30)
  return age_years, age_months

In [ ]:
def create_job_info():
  job = faker.job()
  company = faker.company()
  address = faker.address()
  job_data = {'job_name': job, 'job_company': company, 'job_address': address}
  return job_data

In [ ]:
def create_income_group():
  income_groups = [40000, 80000, 120000, 160000, 200000, 240000, 270000, 300000]
  income_choice = random.choice(income_groups)
  return income_choice

In [ ]:
def create_credit_card_info():
  card_types = ['amex', 'diners', 'discover', 'jcb', 'jcb15', 'jcb16', 'maestro', 'mastercard', 'visa', 'visa13', 'visa16', 'visa19']
  card_choice = random.choice(card_types)
  card_provider = faker.credit_card_provider(card_choice)
  credit_card_number = faker.credit_card_number(card_choice)
  credit_card_security_code = faker.credit_card_security_code(card_choice)
  credit_card_expire = faker.credit_card_expire()
  cc_data = {'card_choice': card_choice, 'card_provider': card_provider, 'credit_card_number': credit_card_number, 
             'credit_card_security_code': credit_card_security_code, 'credit_card_expire': credit_card_expire}
  return cc_data

In [ ]:
def kid_profile(profile_type, family_name, family_id, mom_age_years):
  faimly_data = None
  profile_uuid = str(uuid.uuid4())
  if profile_type in ['m', 'M']:    
    first_name = faker.first_name_male()
  else:
    first_name = faker.first_name_female()
  dob = faker.date_of_birth(maximum_age=(mom_age_years - 18), minimum_age=1,)
  age_years, age_months = get_age_years_month(dob)
  phone_number = faker.phone_number()
  ssn = faker.ssn()                                       # Social Security Number   (ssn)
  faimly_data = {'profile_uuid': profile_uuid, 'family_id': family_id, 'family_name': family_name, 'first_name': first_name, 
                  'dob': dob, 'age_years':age_years,'ssn': ssn, 'marital_status': 'single'}
  return faimly_data

In [ ]:
def add_kids_to_family(family_name, family_id, mom_dob):
  kids_group = random.randint(0, 3)           #Returns any random integer from 0 to 3
  all_kids = []
  mom_age_years, mom_age_months = get_age_years_month(mom_dob)
  if kids_group > 0:
    for each_kid in range(kids_group):
      is_male_kid = faker.boolean() # if true male kid or else female kid
      if is_male_kid == 0: 
        kid_data = kid_profile('m', family_name, family_id, mom_age_years) # male                 # calling the 'kid_profile' function
      else:
        kid_data = kid_profile('f', family_name, family_id, mom_age_years) # female
      all_kids.append(kid_data)
  return all_kids

In [ ]:
def add_user_family(profile_type,family_name, family_id, age_years):
  profile_uuid = str(uuid.uuid4())
  family_groups = ['single', 'married']
  family_choice = random.choice(family_groups)
  faimly_data = None  
  if family_choice == 'married':
    if profile_type in ['m', 'M']:
     first_name = faker.first_name_female()
    else:
     first_name = faker.first_name_male()

    dob = faker.date_of_birth(maximum_age=age_years +5, minimum_age=21)
    age_years, age_months = get_age_years_month(dob)
    phone_number = faker.phone_number()
    ssn = faker.ssn()
     
    faimly_data = {'profile_uuid': profile_uuid, 'family_id': family_id, 'family_name': family_name, 'first_name': first_name, 'dob': dob,
                 'age_years':age_years,'phone_number': phone_number, 'ssn': ssn,'marital_status': family_choice }
  return faimly_data

In [ ]:
def create_user_profile(profile_type):
  profile_uuid = str(uuid.uuid4())
  family_id = faker.lexify(text='??????????') # Family ID
  family_name = faker.last_name()
  if profile_type in ['m', 'M']:    
    first_name = faker.first_name_male()
  else:
    first_name = faker.first_name_female()
    
  dob = faker.date_of_birth(maximum_age=80, minimum_age=21)
  age_years, age_months = get_age_years_month(dob)
  phone_number = faker.phone_number()
  ssn = faker.ssn()
  family_data = {'profile_uuid': profile_uuid, 'family_id': family_id, 'family_name': family_name, 'first_name': first_name, 'dob': dob,
                 'age_years':age_years, 'phone_number': phone_number, 'ssn': ssn}

  # Adding Income Group
  income_group = create_income_group()
  family_data['income_group'] = income_group    # adding 'income_group' as new key in family_data dictionary
  # Adding Job Info
  job_info = create_job_info()      # adding all the return values from the create_job_info() function as the keys +values in the family_data dictionary
  family_data.update(job_info)     
  # Adding Credit Card Info
  cc_info = create_credit_card_info()    # similarly adding the cc_info as the previously using .update method
  family_data.update(cc_info)
  ## Adding User Family            
  user_family_data = add_user_family(profile_type,family_name, family_id, age_years)
  if user_family_data is not None:
    family_data['marital_status'] = 'married'
  else:
    family_data['marital_status'] = 'single'
  # Adding Kids
  kids_list = []
  if age_years >= 25 and user_family_data is not None:
    kids_list = add_kids_to_family(family_name, family_id, user_family_data['dob'])
  return family_data, user_family_data, kids_list

In [ ]:
family_data, user_family_data, kids_list = create_user_profile('M')

print(family_data)
print(user_family_data)
print(kids_list)
print('Kids Count :' ,len(kids_list))

{'profile_uuid': '7b0a7d04-3329-43ef-8e1c-a6ac20de7eaf', 'family_id': 'KgnGsHwxWP', 'family_name': 'Palmer', 'first_name': 'Randy', 'dob': datetime.date(1990, 8, 29), 'age_years': 31, 'phone_number': '295-613-0350', 'ssn': '171-60-9212', 'income_group': 80000, 'job_name': 'Research scientist (life sciences)', 'job_company': 'Walker Group', 'job_address': '72938 Bullock Radial Suite 579\nLake Joseph, ND 62904', 'card_choice': 'discover', 'card_provider': 'Discover', 'credit_card_number': '6585790882070259', 'credit_card_security_code': '816', 'credit_card_expire': '01/24', 'marital_status': 'married'}
{'profile_uuid': '62e69999-6853-4657-a56b-a76547ef8ea0', 'family_id': 'KgnGsHwxWP', 'family_name': 'Palmer', 'first_name': 'Bridget', 'dob': datetime.date(1993, 8, 26), 'age_years': 28, 'phone_number': '(274)453-4798x54066', 'ssn': '322-75-0535', 'marital_status': 'married'}
[{'profile_uuid': '18e975e2-31ac-45af-a2d9-ae5cb5e2c72c', 'family_id': 'KgnGsHwxWP', 'family_name': 'Palmer', 'first

#Vehicle Data

In [ ]:
!pip install faker_vehicle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 5.4 MB/s 


In [ ]:
from faker_vehicle import VehicleProvider
faker.add_provider(VehicleProvider)

In [ ]:
def create_vehicle_data(family_id):           # relating Vehicle Data to a particullar family_id
  vehicle_object = faker.vehicle_object()
  vehicle_license = faker.pystr_format(string_format="???? ###")
  vehicle_vin = faker.pystr_format(string_format="#??###??????######")
  vehicle_data = {'vehicle_category': vehicle_object['Category'], 'vehicle_maker': vehicle_object['Make'], 
                  'vehicle_model': vehicle_object['Model'], 'vehicle_year': vehicle_object['Year'], 
                  'vehicle_license': vehicle_license, 'vehicle_vin': vehicle_vin,
                  'family_id': family_id}
  return vehicle_data

In [ ]:
vehicle_data = create_vehicle_data('xx')
vehicle_data

{'family_id': 'xx',
 'vehicle_category': 'SUV',
 'vehicle_license': 'peal 221',
 'vehicle_maker': 'Toyota',
 'vehicle_model': 'Sequoia',
 'vehicle_vin': '6nA185IkTJAB517045',
 'vehicle_year': 2011}

# Address or Location Data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/KislayKaran/Walmart_Supermarket/main/Required_CSV_files/04_c_wm_stores_single_precision_(created_csv).csv')


In [ ]:
df = df[['state', 'city', 'zip_code', 'latitude', 'longitude']]
df.head()

,state,city,zip_code,latitude,longitude
0,LA,Abbeville,70510,30.0,-92.1
1,MD,Aberdeen,21001,39.5,-76.2
2,NC,Aberdeen,28315,35.2,-79.4
3,SD,Aberdeen,57401,45.5,-98.4
4,WA,Aberdeen,98520,47.0,-123.8


In [ ]:
def get_walmart_loc_by_state(state):  
  if state is not None:
    df_local = df[df['state'] == state]
    if len(df_local) > 0:
      local_location = df_local.sample(1).to_dict(orient='records')
      if local_location is not None and len(local_location) > 0:
        return local_location[0]
  return None

  

In [ ]:
#Suppose
store = get_walmart_loc_by_state('NY')
store

{'city': 'Lancaster',
 'latitude': 42.9,
 'longitude': -78.7,
 'state': 'NY',
 'zip_code': 14043}

In [ ]:
def generate_user_location(family_id, state):
  street_address = faker.street_address()
  selected_location = get_walmart_loc_by_state(state)
  state = selected_location['state']
  city = selected_location['city']
  zip_code = selected_location['zip_code']
  latitude = selected_location['latitude']
  longitude = selected_location['longitude']
  #print("latitude : " + str(latitude) + ' longitude: ' + str(longitude))
  random_6_digits_float = round(random.uniform(0.01, 0.09), 6)
  if latitude > 0:
    latitude = latitude + random_6_digits_float
  else:
    latitude = latitude - random_6_digits_float
  latitude = round(latitude,6)
  random_6_digits_float = round(random.uniform(0.01, 0.09), 6)
  if longitude > 0:
    longitude = longitude + random_6_digits_float
  else:
    longitude = longitude - random_6_digits_float
  longitude = round(longitude,6)
  #print("latitude : " + str(latitude) + ' longitude: ' + str(longitude))
  location_data = {'street_address': street_address, 'city': city, 'state': state, 'zip_code': zip_code, 
                   'longitude': longitude, 'latitude': latitude, 'family_id': family_id}
  return location_data

#Combining User Profile and family with location data

In [ ]:
user_profile, user_family_data, kids_list = create_user_profile('m')
location_data = generate_user_location(user_profile['family_id'], 'CA')
vehicle_data = create_vehicle_data(user_profile['family_id'])

In [ ]:
print(user_profile)
print(user_family_data)
print(kids_list)
print(location_data)
print(vehicle_data)

{'profile_uuid': 'db50c86c-744b-41b6-abba-a23d161916b9', 'family_id': 'ZOHGtiwiaE', 'family_name': 'Perez', 'first_name': 'James', 'dob': datetime.date(1994, 3, 23), 'age_years': 28, 'phone_number': '(757)740-8767', 'ssn': '110-68-2197', 'income_group': 80000, 'job_name': 'Water engineer', 'job_company': 'Fowler Ltd', 'job_address': 'USCGC Taylor\nFPO AA 52427', 'card_choice': 'visa', 'card_provider': 'VISA 16 digit', 'credit_card_number': '4354179721696496', 'credit_card_security_code': '498', 'credit_card_expire': '10/30', 'marital_status': 'married'}
{'profile_uuid': 'e8dc278b-6bef-4920-9702-863ae0cf94a8', 'family_id': 'ZOHGtiwiaE', 'family_name': 'Perez', 'first_name': 'Brandy', 'dob': datetime.date(1999, 10, 20), 'age_years': 22, 'phone_number': '122.415.5377x7412', 'ssn': '203-59-2994', 'marital_status': 'married'}
[{'profile_uuid': '6b7feb84-06ad-47a6-b3a0-e386168aa1cc', 'family_id': 'ZOHGtiwiaE', 'family_name': 'Perez', 'first_name': 'Daniel', 'dob': datetime.date(2020, 3, 26),

#Creating Record(s) into DataFrame

In [ ]:
all_states = df['state'].unique().tolist()
print(len(all_states))
print(all_states)

52
['LA', 'MD', 'NC', 'SD', 'WA', 'TX', 'VA', 'MA', 'GA', 'OK', 'IL', 'MI', 'SC', 'OH', 'AL', 'NM', 'CO', 'NY', 'OR', 'MN', 'TN', 'KY', 'PA', 'AR', 'FL', 'IA', 'CA', 'UT', 'NH', 'ID', 'MS', 'AK', 'IN', 'AZ', 'WI', 'KS', 'MO', 'ME', 'NJ', 'CT', 'WV', 'PR', 'NE', 'VT', 'MT', 'ND', 'DE', 'NV', 'WY', 'RI', 'HI', 'DC']


In [ ]:
user_profile_records = []
user_address_records = []
user_vehicle_records = []
total_users = 0
record_count = 100
for state in all_states: # ['CA', 'WA', 'OR']:
  print('Generating ' + str(record_count) + ' records for state - ' + state)
  for i in range(record_count):
    user_profile, user_family_data, kids_list = create_user_profile('m')
    user_profile_records.append(user_profile)
    total_users = total_users + 1
    if user_family_data is not None:
      user_profile_records.append(user_family_data)
      total_users = total_users + 1
    if len(kids_list) > 0:
      for kid in kids_list:
        total_users = total_users + 1
        user_profile_records.append(kid)
    location_data = generate_user_location(user_profile['family_id'], state)
    user_address_records.append(location_data)
    user_vehicle_records.append(create_vehicle_data(user_profile['family_id']))
    if user_profile['marital_status'] == 'married':
      user_vehicle_records.append(create_vehicle_data(user_profile['family_id']))

print("Total Profiles: ", total_users)


Generating 100 records for state - LA
Generating 100 records for state - MD
Generating 100 records for state - NC
Generating 100 records for state - SD
Generating 100 records for state - WA
Generating 100 records for state - TX
Generating 100 records for state - VA
Generating 100 records for state - MA
Generating 100 records for state - GA
Generating 100 records for state - OK
Generating 100 records for state - IL
Generating 100 records for state - MI
Generating 100 records for state - SC
Generating 100 records for state - OH
Generating 100 records for state - AL
Generating 100 records for state - NM
Generating 100 records for state - CO
Generating 100 records for state - NY
Generating 100 records for state - OR
Generating 100 records for state - MN
Generating 100 records for state - TN
Generating 100 records for state - KY
Generating 100 records for state - PA
Generating 100 records for state - AR
Generating 100 records for state - FL
Generating 100 records for state - IA
Generating 1

In [ ]:
user_df = pd.DataFrame(user_profile_records)
address_df = pd.DataFrame(user_address_records)
vehicle_df = pd.DataFrame(user_vehicle_records)


In [ ]:
user_df.to_csv('05_c_user_profile_[(created_csv) using_faker].csv')

In [ ]:
address_df.to_csv('06_c_user_address_[(created_csv) using_faker].csv')

In [ ]:
vehicle_df.to_csv('07_c_user_vehicle_[(created_csv) using_faker].csv')

In [ ]:
user_df.head()

,profile_uuid,family_id,family_name,first_name,dob,age_years,phone_number,ssn,income_group,job_name,job_company,job_address,card_choice,card_provider,credit_card_number,credit_card_security_code,credit_card_expire,marital_status
0,ebd20c5e-2eee-4bc7-9003-b9d713695399,cXOKfEmuaP,Fuller,Jeremiah,1977-12-09,44,564-235-6420x2707,093-82-0703,40000.0,Horticultural therapist,Martin LLC,"32063 Jessica Walks\nSouth Christopherstad, ID...",jcb16,JCB 16 digit,3501928094619764,220,02/27,married
1,a6d86078-9ed1-4863-a9df-9fde1a52f8c9,cXOKfEmuaP,Fuller,Teresa,1987-06-19,35,690-786-5666x772,010-11-9469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,married
2,fceeffca-81a1-4111-80d2-8b9e494f5ee5,cXOKfEmuaP,Fuller,Brandon,2006-09-02,15,NaN,643-53-3672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single
3,067e6fd5-6db9-40cf-a59b-afaa06d8bc69,cXOKfEmuaP,Fuller,Krista,2015-08-12,6,NaN,325-64-1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single
4,2ce1e78a-0cdf-4889-9990-91bf5a5ff2b5,WyUHMYshda,Harmon,Robert,1964-06-27,58,379-473-2357x68927,728-20-5617,270000.0,Personal assistant,Barton PLC,USNV Gordon\nFPO AP 83113,visa,VISA 16 digit,4085846699561771,810,01/32,single


In [ ]:
 len(user_df. index)          # total profiles Generated

11390

In [ ]:
address_df.head()

,street_address,city,state,zip_code,longitude,latitude,family_id
0,67582 Diaz Parkway Apt. 713,New Orleans,LA,70114,-90.046132,29.945745,cXOKfEmuaP
1,06898 Gutierrez Motorway,Slidell,LA,70458,-89.861673,30.351911,WyUHMYshda
2,62930 Murphy Forges,West Monroe,LA,71291,-92.242020,32.510795,bFfcQwnSUD
3,157 Duncan Route Apt. 996,Slidell,LA,70461,-89.713286,30.361041,JAKIJrkNwq
4,146 Rodriguez Plaza,Lake Charles,LA,70611,-93.284018,30.383500,oShKyaaEDW


In [ ]:
 len(address_df. index)     # Total address generated will be 52 (States) x 100 (records each)

5200

In [ ]:
vehicle_df.head()

,vehicle_category,vehicle_maker,vehicle_model,vehicle_year,vehicle_license,vehicle_vin,family_id
0,"Sedan, Hatchback",Kia,Spectra,2002,tDdJ 279,3mR223XucqyK567941,cXOKfEmuaP
1,Pickup,Dodge,Ram 2500 Crew Cab,2010,dyor 447,6QE077lTgztt431305,cXOKfEmuaP
2,SUV,Volvo,XC60,2016,npNc 433,3qQ293tdxwRT438592,WyUHMYshda
3,Van/Minivan,Nissan,Quest,1996,fMCv 438,3Gz059LtFIoX413603,bFfcQwnSUD
4,Coupe,Scion,tC,2012,qiQZ 776,2Ga827Ekpoaf752678,bFfcQwnSUD


In [ ]:
 len(vehicle_df. index) 

7812